# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [15]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal
from prettytable import PrettyTable

ModuleNotFoundError: No module named 'cassandra'

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

#### Import Apache Cassandra python package

In [ ]:
import cassandra

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
try:
    from cassandra.cluster import Cluster
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except exception as e:
    print(e)

#### Create Keyspace

In [ ]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
        print(e)

def Execute_Query(query):
    try:
        return session.execute(query)
    except Exception as e:
        print(e)

def Execute_Query_Param(query,param):
    try:
        return session.execute(query,param)
    except Exception as e:
        print(e)
        
def Process_CSV(file,func):
    """
    Process all the data executing the given func for every *.json file of the given filepath
    :param filepath: The file path
    :return:None
    """
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)
        for line in csvreader:
            func(line)

#### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

**Table Structure**

According to above query, we need to create primary key as session_id & item_in_session. And add only 3 more column : artist_name, song_title & song_length

In [ ]:
Execute_Query('''CREATE TABLE IF NOT EXISTS session_library (
                   session_id int, 
                   item_in_session int, 
                   artist_name text, 
                   song_title text, 
                   song_length decimal, 
                   PRIMARY KEY (session_id, item_in_session))''')
                    

**Data insertion**

In this table, we will do only 3 transforms:

* session_id should be an int
* item_in_session should be an int
* song_length should be a decimal

In [ ]:
def insert_session_library(line):
    query = "INSERT INTO session_library(session_id, item_in_session, artist_name, song_title, song_length)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    queryParameters = (int(line[8]), int(line[3]), line[0], line[9], Decimal(line[5]))
    
    return Execute_Query_Param(query,queryParameters)

Process_CSV('event_datafile_new.csv', func=insert_session_library)

**Get appropriate data using select query**

In [ ]:
query = "SELECT * FROM session_library WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
t = PrettyTable(['sessionId', 'item number in session', 'artist', 'song title','length of the song'])
for row in rows:
    t.add_row([row.session_id, row.item_in_session, row.artist_name, row.song_title, row.song_length])
print(t)

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

**Table Structure**

According to above query, we need to create primary key as userid & session_id. And set clustering column as item_in_session. Add other column like : artist_name, song_title, first_name & last_name.

In [9]:
Execute_Query('''CREATE TABLE IF NOT EXISTS user_library (
                   userid int, 
                   session_id int,
                   item_in_session int, 
                   artist_name text, 
                   song_title text, 
                   first_name text,
                   last_name text,
                   PRIMARY KEY ((userid, session_id), item_in_session))''')                    

NameError: name 'Execute_Query' is not defined

**Data insertion**

In this table, we will do only 3 transforms:

* userid should be an int
* session_id should be an int
* item_in_session should be an int

In [10]:
def insert_to_user_library(line):
    query = "INSERT INTO user_library(userid, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
    queryParameters = (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])
    
    return Execute_Query_Param(query,queryParameters)

Process_CSV('event_datafile_new.csv', func=insert_to_user_library)
                    

NameError: name 'Process_CSV' is not defined

**Get appropriate data using select query**

In [ ]:
query = "SELECT * FROM user_library WHERE session_id=182 AND userid=10"
try:
    rows = session.execute(query)
except Exception as e:
    printa(e)
    
t = PrettyTable(['item num in session', 'artist', 'song title','first name','last name'])
for row in rows:
    t.add_row([row.item_in_session, row.artist_name, row.song_title, row.first_name, row.last_name])
print(t)

## 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

**Table Structure**

According to above query, we need to create primary key as song_title & userid. And add only 3 more column : artist_name, first_name & last_name.

In [11]:
Execute_Query('''CREATE TABLE IF NOT EXISTS user_song_library (
                   userid int, 
                   song_title text, 
                   artist_name text, 
                   first_name text,
                   last_name text,
                   PRIMARY KEY (song_title, userid))''')                  

NameError: name 'Execute_Query' is not defined

**Data insertion**

In this table, we will do only 3 transforms:

* userid should be an int

In [ ]:
def insert_to_user_song_library(line):
    query = "INSERT INTO user_song_library(userid, song_title, artist_name, first_name, last_name)"
    query = query + " VALUES (%s, %s, %s, %s, %s)"
    queryParameters = (int(line[10]), line[9], line[0], line[1], line[4])
    
    return Execute_Query_Param(query,queryParameters)

Process_CSV('event_datafile_new.csv', func=insert_to_user_song_library)
          

**Get appropriate data using select query**

In [12]:
query = "SELECT * FROM user_song_library WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
t = PrettyTable(['first name','last name', 'artist', 'song title'])
for row in rows:
    t.add_row([ row.first_name, row.last_name, row.artist_name, row.song_title])
print(t)

name 'session' is not defined


NameError: name 'PrettyTable' is not defined

### Drop the tables before closing out the sessions

In [13]:
try:
    rows = session.execute("drop table user_library")
except Exception as e:
    print(e)
try:
    rows = session.execute("drop table session_library")
except Exception as e:
    print(e)
try:
    rows = session.execute("drop table user_song_library")
except Exception as e:
    print(e)

name 'session' is not defined
name 'session' is not defined
name 'session' is not defined


### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()

NameError: name 'session' is not defined